In [1]:
import openai
import os

In [9]:
openai.api_key = '' # find in /xnetwork-backend/xnetwork/.env

In [6]:
# %pip install openai==0.28

In [11]:
MODEL = "gpt-3.5-turbo"
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Knock knock."},
        {"role": "assistant", "content": "Who's there?"},
        {"role": "user", "content": "Orange."},
    ],
    temperature=0,
)

response

<OpenAIObject chat.completion id=chatcmpl-8jtwTNh76YYthdSgatyEqSpoEOAWk at 0x7fb5d1922e50> JSON: {
  "id": "chatcmpl-8jtwTNh76YYthdSgatyEqSpoEOAWk",
  "object": "chat.completion",
  "created": 1705950493,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Orange who?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 35,
    "completion_tokens": 3,
    "total_tokens": 38
  },
  "system_fingerprint": null
}

In [16]:
import openai

MODEL = "gpt-3.5-turbo"

def extract_search_criteria(user_input):
    """
    Extracts movie search criteria from the user's input.
    
    :param user_input: The user's message as a string.
    :return: Dictionary with extracted information.
    """
    prompt = f"Extract movie search criteria from the following user input: '{user_input}'"
    
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=[{"role": "system", "content": prompt}],
        temperature=0,
    )

    extracted_info = response.get('choices', [])[0].get('message', {'content': ''})['content'].strip()

    # Assuming the model returns a comma-separated list of key-value pairs
    criteria = {}
    for item in extracted_info.split(','):
        key_value = item.split(':')
        if len(key_value) == 2:
            criteria[key_value[0].strip()] = key_value[1].strip()

    return criteria


def generate_query_text(movie_search_info):
    """
    Generates a search query based on the accumulated movie search criteria.
    
    :param movie_search_info: Dictionary containing the search criteria.
    :return: A query string.
    """
    query_parts = []
    for key, value in movie_search_info.items():
        query_parts.append(f"{key}:{value}")
    return " AND ".join(query_parts)


def chat_with_gpt():
    messages = [{"role": "system", "content": "You are a helpful assistant specializing in movie searches."}]
    movie_search_info = {}  # Dictionary to store movie search criteria

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["quit", "exit"]:
            break

        messages.append({"role": "user", "content": user_input})

        response = openai.ChatCompletion.create(
            model=MODEL,
            messages=messages,
            temperature=0,
        )

        assistant_response = response['choices'][0]['message']['content']
        print(f"Assistant: {assistant_response}")

        # Here you would add logic to extract movie search criteria from the conversation
        # For example, identifying genres, actors, etc., mentioned by the user
        movie_search_info.update(extract_search_criteria(user_input))
        
        # Once enough information is gathered, formulate the search query
        query_text = generate_query_text(movie_search_info)

        print("movie_search_info: ")
        print(movie_search_info)

        print(f'query_text: {query_text}')

        # Optionally, make an API call to your movie search system with the query_text
        # movie_results = call_movie_search_api(query_text)

        # Add assistant's response to the conversation history
        messages.append({"role": "assistant", "content": assistant_response})

if __name__ == "__main__":
    chat_with_gpt()


You:  adventure


Assistant: Great! Adventure movies are always exciting. Can you please provide me with more specific information? For example, do you have any preferences for the time period, setting, or any particular actors or directors you enjoy?
movie_search_info: 
{'Genre': 'Adventure'}
query_text: Genre:Adventure


You:  titanic


Assistant: Ah, "Titanic" is a classic adventure movie! Released in 1997, it was directed by James Cameron and starred Leonardo DiCaprio and Kate Winslet. The film is set on the ill-fated maiden voyage of the RMS Titanic in 1912. It combines elements of romance, drama, and action as it follows the love story between Jack Dawson (DiCaprio) and Rose DeWitt Bukater (Winslet) amidst the backdrop of the ship's tragic sinking.

If you enjoyed "Titanic," you might also like other adventure movies such as "Indiana Jones" series, "Pirates of the Caribbean" series, "The Mummy" series, or "Jurassic Park." Let me know if you would like more recommendations or if there's anything else I can assist you with!
movie_search_info: 
{'Genre': 'Adventure', 'Movie title': "'titanic'"}
query_text: Genre:Adventure AND Movie title:'titanic'


You:  ok, indiana jones


Assistant: Great choice! The "Indiana Jones" series is a beloved adventure franchise. Created by George Lucas and directed by Steven Spielberg, the series follows the adventures of the archaeologist and adventurer Indiana Jones, played by Harrison Ford.

Here are the movies in the series:

1. "Raiders of the Lost Ark" (1981): In this film, Indiana Jones embarks on a quest to find the Ark of the Covenant before the Nazis can use its power for their own gain.

2. "Indiana Jones and the Temple of Doom" (1984): Set in 1935, this film sees Indiana Jones trying to rescue enslaved children and recover a sacred stone from a cult in India.

3. "Indiana Jones and the Last Crusade" (1989): In this installment, Indiana Jones teams up with his father, played by Sean Connery, to find the Holy Grail before the Nazis do.

4. "Indiana Jones and the Kingdom of the Crystal Skull" (2008): Set in the 1950s, this film follows Indiana Jones as he searches for a mythical crystal skull while facing Soviet agen

You:  exit
